# Install OpenShift client

In [19]:
%%bash
curl -o oc.tar.gz -L https://github.com/openshift/origin/releases/download/v3.11.0/openshift-origin-client-tools-v3.11.0-0cbc58b-linux-64bit.tar.gz
tar xzf oc.tar.gz
cp openshift-origin-client-tools-v3.11.0-0cbc58b-linux-64bit/oc ~/../bin/oc


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   654    0   654    0     0   1778      0 --:--:-- --:--:-- --:--:--  1777
100 53.8M  100 53.8M    0     0   9.9M      0  0:00:05  0:00:05 --:--:-- 11.3M
tar: Ignoring unknown extended header keyword `LIBARCHIVE.xattr.security.selinux'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.xattr.security.selinux'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.xattr.security.selinux'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.xattr.security.selinux'


# Check the Service Account

In [42]:
%%bash
oc whoami


system:serviceaccount:vpavlin-jupyterhub:jupyter-tensorflow


# List templates

You should see templates imported from [odh-tensorflow-jobs](https://github.com/vpavlin/odh-tensorflow-jobs/tree/master/openshift) repository

In [22]:
%%bash
oc get templates

NAME                                    DESCRIPTION                                                                        PARAMETERS    OBJECTS
jupyter-notebook-workspace-tensorflow   Template for deploying Tensorflow enable Jupyter Notebook images with persist...   4 (all set)   7
odh-config                              Template to configure basic components of ODH ML flow                              3 (2 blank)   1
odh-tensorflow-serving                  Template to serve models using tensorflow                                          5 (all set)   2
odh-tensorflow-training                 Template to train models using tensorflow                                          9 (all set)   1


# List parameters for the training template

`oc process` command allows you to list configurable parameters for a given template. We'll look at the training job tempalte first.

In [21]:
%%bash
oc process odh-tensorflow-training --parameters

NAME                    DESCRIPTION                                                                                              GENERATOR           VALUE
APP_NAME                Short name of your application (to be used in OpenShift artifact names)                                                      demo
TRAINING_STEPS          Number of training steps to perform                                                                                          10000
INPUT_DATA_LOCATION     Location of input data in form of 's3://BUCKET/path/', path is used as prefix to lookup the data in S3                       s3://MY-BUCKET/data/
OUTPUT_MODEL_LOCATION   Location where the resulting model will be stored                                                                            s3://MY-BUCKET/model-out/
MEMORY                  Memory limit to be assigned to the job                                                                                       2Gi
CPU                     Limit for numb

# Deploy the training job

Configure all necessary parameters from above and pipe the `oc process` output to `oc apply` command to submit the job to OpenShift

In [28]:
%%bash
oc process odh-tensorflow-training \
    -p APP_NAME=from-jupyter \
    -p TRAINING_STEPS=2000 \
    -p INPUT_DATA_LOCATION="s3://VPAVLIN-SF/intect-data" \
    -p OUTPUT_MODEL_LOCATION="s3://VPAVLIN-SF/output-model" \
    -p MEMORY="9Gi" \
    -p CPU="9" \
        | oc apply -f -

job.batch/odh-tensorflow-training-from-jupyter created


# View logs

You can review the job execution by (re)running the below `oc logs` command

In [30]:
%%bash
oc logs job.batch/odh-tensorflow-training-from-jupyter

Cloning into 'app_code'...
Using context dir training
Processing /opt/app-root/src/app_code/pyodh
  Cloning https://github.com/CermakM/intect to /tmp/pip-5ofw11m8-build
  Running setup.py install for pyodh: started
    Running setup.py install for pyodh: finished with status 'done'
  Running setup.py install for names: started
    Running setup.py install for names: finished with status 'done'
  Running setup.py install for pyyaml: started
    Running setup.py install for pyyaml: finished with status 'done'
  Running setup.py install for networkx: started
    Running setup.py install for networkx: finished with status 'done'
  Running setup.py install for toolz: started
    Running setup.py install for toolz: finished with status 'done'
  Running setup.py install for intect: started
    Running setup.py install for intect: finished with status 'done'
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' com

# List parameters for Tensorflow serving template

Once the training job is completed, we can deploy the serving endpoint - see the parametrs below.

In [32]:
%%bash
oc process odh-tensorflow-serving --parameters

NAME                   DESCRIPTION                                                               GENERATOR           VALUE
APP_NAME               Short name of your application (to be used in OpenShift artifact names)                       demo
INPUT_MODEL_LOCATION   Location where the resulting model will be stored                                             s3://MY-BUCKET/model-out/
MODEL_NAME             Name of the model                                                                             mnist
MEMORY                 Memory limit to be assigned to the job                                                        1Gi
CPU                    Limit for number of cores to assign to the job                                                1


# Deploy Tensorflow serving endpoint

In [33]:
%%bash
oc process odh-tensorflow-serving \
    -p APP_NAME=from-jupyter \
    -p INPUT_MODEL_LOCATION="s3://VPAVLIN-SF/output-model" \
    -p MODEL_NAME="intect" \
        | oc apply -f -

deploymentconfig.apps.openshift.io/odh-tensorflow-serving-from-jupyter created
service/odh-tensorflow-serving-from-jupyter created


In [34]:
%%bash
oc logs deploymentconfig.apps.openshift.io/odh-tensorflow-serving-from-jupyter

+ python prep.py
+ tensorflow_model_server --port=6006 --model_name=intect --model_base_path=/opt/app-root/src/
2018-12-12 11:57:21.352995: I tensorflow_serving/model_servers/main.cc:157] Building single TensorFlow model file config:  model_name: intect model_base_path: /opt/app-root/src/
2018-12-12 11:57:21.353275: I tensorflow_serving/model_servers/server_core.cc:462] Adding/updating models.
2018-12-12 11:57:21.353297: I tensorflow_serving/model_servers/server_core.cc:517]  (Re-)adding model: intect
2018-12-12 11:57:21.453766: I tensorflow_serving/core/basic_manager.cc:739] Successfully reserved resources to load servable {name: intect version: 1544613698}
2018-12-12 11:57:21.453837: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: intect version: 1544613698}
2018-12-12 11:57:21.453855: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: intect version: 1544613698}
2018-12-12 11:57:21.453900: I external/org_tensorf

# Testing the predictions

As the training job uses [intect]() tool, we can use `intect-client` to call to our model server. Run the following cell to install the client.

In [44]:
%%bash
pip install git+https://github.com/CermakM/intect

  Cloning https://github.com/CermakM/intect to /tmp/pip-req-build-fc6y2tlp


# Calling prediction endpoint

We need to configure the server name - you can see the name of the deployed service above as `service/odh-tensorflow-serving-from-jupyter` - we'll use the part after slash. The model name needs to match the name defined in `MODEL_NAME` parameter above.

You can use [2.png](https://github.com/vpavlin/odh-tensorflow-jobs/blob/master/2.png) to test the prediction.

In [46]:
%%bash
intect-client --host odh-tensorflow-serving-from-jupyter --port 6006 --model_name intect --images 2.png

outputs {
  key: "output"
  value {
    dtype: DT_STRING
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 1
      }
    }
    string_val: "2"
  }
}
model_spec {
  name: "intect"
  version {
    value: 1544613698
  }
  signature_name: "prediction"
}

outputs {
  key: "output"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
    }
    float_val: 0.9523658752441406
  }
}
model_spec {
  name: "intect"
  version {
    value: 1544613698
  }
  signature_name: "confidence"
}



/opt/app-root/lib/python3.6/site-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/opt/app-root/lib/python3.6/site-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):
